In [6]:
import os
import pandas as pd
import re
import numpy as np

In [7]:
income_path = '../data/income/'
processed_path = '../data/processed/'

In [8]:
income = pd.read_csv(income_path + 'income.csv')
states_name = pd.read_csv(income_path + 'states_name.csv')
population = pd.read_csv(income_path + 'population.csv')
time_series_mobility = pd.read_csv(processed_path + 'time_series_mobility.csv')

In [9]:
population

,FIPS,State,Area_Name,POP_ESTIMATE_2018
0,0,US,United States,"327,167,434"
1,1000,AL,Alabama,"4,887,871"
2,1001,AL,Autauga County,"55,601"
3,1003,AL,Baldwin County,"218,022"
4,1005,AL,Barbour County,"24,881"
...,...,...,...,...
3189,56037,WY,Sweetwater County,"43,051"
3190,56039,WY,Teton County,"23,081"
3191,56041,WY,Uinta County,"20,299"
3192,56043,WY,Washakie County,"7,885"


In [10]:
income_split = np.split(income, income[income.isnull().all(1)].index)[1:]
income_clean = pd.DataFrame()
for state in income_split:
    state_name = state['County'][1:2].values
    state['state'] = state_name[0]
    if state_name == 'District of Columbia':
        income_clean = income_clean.append(state[1:2])
    else: 
        income_clean = income_clean.append(state[2:])

In [11]:
income_clean['state'] = income_clean['state'].str.lower()
income_clean['county'] = income_clean['County'].str.lower()
income_clean['income_2018'] = income_clean['2,018']
income_clean = income_clean.drop(['County','2,018'], axis=1)
income_clean = income_clean[['county','state','income_2018']]

In [12]:
income_clean['income_2018'] = income_clean['income_2018'].map(lambda x: int(re.sub(',','', x)))

In [13]:
income_clean

,county,state,income_2018
2,autauga,alabama,41618
3,baldwin,alabama,45596
4,barbour,alabama,35199
5,bibb,alabama,30254
6,blount,alabama,34976
...,...,...,...
3176,sweetwater,wyoming,53145
3177,teton,wyoming,251728
3178,uinta,wyoming,40280
3179,washakie,wyoming,48184


In [14]:
states = population['State']

In [15]:
# find index of 
index_list = []
for state in states.unique():
    index_list.append(states[(states == state)].index[0])

In [16]:
population = population.drop(index_list)

In [17]:
population['Area_Name'] = population['Area_Name'].map(lambda x: re.sub(' County| City','', x)).str.lower()

In [18]:
dictionary = {short:long for (short,long) in zip(states_name['Code'], states_name['State'])}
population['State'] = population['State'].map(dictionary).str.lower()

In [19]:
population = population[['FIPS','State', 'Area_Name', 'POP_ESTIMATE_2018']]
population.columns = ['fips','state','county', 'pop_2018']

In [20]:
population['pop_2018'] = population['pop_2018'].map(lambda x: int(re.sub(',','', x)))

In [21]:
# merge income
income_combined = pd.merge(time_series_mobility, income_clean, on=['state','county'])

In [23]:
income_combined

,Unnamed: 0,date,county,state,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,...,wetBulbMax,first_date,days_since_10_cases,grocery_and_pharmacy_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018
0,0,2020-01-21,snohomish,washington,53061,1,0,7.0,71.0,100.0,...,44.1,2020-03-05,-44,NaN,NaN,NaN,NaN,NaN,NaN,55888
1,1,2020-01-22,snohomish,washington,53061,1,0,31.0,74.0,100.0,...,44.6,2020-03-05,-43,NaN,NaN,NaN,NaN,NaN,NaN,55888
2,2,2020-01-23,snohomish,washington,53061,1,0,75.0,93.0,100.0,...,50.7,2020-03-05,-42,NaN,NaN,NaN,NaN,NaN,NaN,55888
3,3,2020-01-24,snohomish,washington,53061,1,0,15.0,57.0,95.0,...,48.3,2020-03-05,-41,NaN,NaN,NaN,NaN,NaN,NaN,55888
4,4,2020-01-25,snohomish,washington,53061,1,0,17.0,72.0,100.0,...,48.0,2020-03-05,-40,NaN,NaN,NaN,NaN,NaN,NaN,55888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53624,54946,2020-04-15,saline,nebraska,31151,1,0,0.0,1.0,12.0,...,42.8,2020-04-21,-6,NaN,NaN,NaN,NaN,NaN,NaN,43763
53625,54947,2020-04-16,saline,nebraska,31151,3,0,45.0,88.0,100.0,...,34.5,2020-04-21,-5,NaN,NaN,NaN,NaN,NaN,NaN,43763
53626,54948,2020-04-17,saline,nebraska,31151,4,0,0.0,35.0,100.0,...,40.9,2020-04-21,-4,NaN,NaN,NaN,NaN,NaN,NaN,43763
53627,54949,2020-04-18,saline,nebraska,31151,6,0,0.0,9.0,69.0,...,55.4,2020-04-21,-3,NaN,NaN,NaN,NaN,NaN,NaN,43763


In [93]:
time_series_all  = pd.merge(income_combined, population, on=['fips'], how='left')

In [94]:
time_series_all.insert(0,'county',time_series_all['county_x'])
time_series_all.insert(0,'state',time_series_all['state_x'])
time_series_all = time_series_all.drop(['Unnamed: 0','county_x','state_x','state_y','county_y'],axis=1)



In [96]:
time_series_all.to_csv(processed_path+'time_series_all.csv',index=False)

In [97]:
df = pd.read_csv(processed_path+'time_series_all.csv')

In [98]:
df

,state,county,date,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,...,first_date,days_since_10_cases,grocery_and_pharmacy_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018
0,washington,snohomish,2020-01-21,53061,1,0,7.0,71.0,100.0,37.4,...,2020-03-05,-44,NaN,NaN,NaN,NaN,NaN,NaN,55888,814901
1,washington,snohomish,2020-01-22,53061,1,0,31.0,74.0,100.0,39.9,...,2020-03-05,-43,NaN,NaN,NaN,NaN,NaN,NaN,55888,814901
2,washington,snohomish,2020-01-23,53061,1,0,75.0,93.0,100.0,44.3,...,2020-03-05,-42,NaN,NaN,NaN,NaN,NaN,NaN,55888,814901
3,washington,snohomish,2020-01-24,53061,1,0,15.0,57.0,95.0,43.9,...,2020-03-05,-41,NaN,NaN,NaN,NaN,NaN,NaN,55888,814901
4,washington,snohomish,2020-01-25,53061,1,0,17.0,72.0,100.0,41.4,...,2020-03-05,-40,NaN,NaN,NaN,NaN,NaN,NaN,55888,814901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53624,nebraska,saline,2020-04-15,31151,1,0,0.0,1.0,12.0,17.4,...,2020-04-21,-6,NaN,NaN,NaN,NaN,NaN,NaN,43763,14350
53625,nebraska,saline,2020-04-16,31151,3,0,45.0,88.0,100.0,22.7,...,2020-04-21,-5,NaN,NaN,NaN,NaN,NaN,NaN,43763,14350
53626,nebraska,saline,2020-04-17,31151,4,0,0.0,35.0,100.0,23.9,...,2020-04-21,-4,NaN,NaN,NaN,NaN,NaN,NaN,43763,14350
53627,nebraska,saline,2020-04-18,31151,6,0,0.0,9.0,69.0,30.1,...,2020-04-21,-3,NaN,NaN,NaN,NaN,NaN,NaN,43763,14350
